In [9]:
import os
import xarray as xr
import pandas as pd
from bokeh.plotting import figure, row
from bokeh.models import HoverTool, Range1d, ColumnDataSource
from bokeh.io import show, output_notebook
from bokeh.transform import factor_cmap
from bokeh.palettes import viridis
from bokeh.models import FactorRange
from bokeh.core.properties import field

output_notebook()

Loading BokehJS ...

## Get the data

In [2]:
data_dir='~/data/allen/'
data_dir = os.path.expanduser(data_dir)
probe_id = "810755797"

In [3]:
output_filepath = os.path.join(data_dir, f"lfp_{probe_id}.zarr")
# Disable mask_and_scale otherwise dtypes are converted to floats.
ds = xr.open_zarr(output_filepath, mask_and_scale=False, consolidated=False)

In [4]:
# start with smaller
crop_t = 100
crop_c = 20
time = ds.lfp.time.data[:crop_t]
channels = ds.lfp.channel.data.astype(str)[:crop_c]
data = ds.lfp[:crop_t, :crop_c].compute().data.T


In [5]:
ds

<xarray.Dataset>
Dimensions:                    (channel: 93, filtering: 93, group_name: 93,
                                id: 93, time: 10715666, local_index: 93,
                                location: 93, probe_horizontal_position: 93,
                                probe_id: 93, probe_vertical_position: 93,
                                valid_data: 93, x: 93, y: 93, z: 93)
Coordinates: (12/14)
  * channel                    (channel) int64 0 1 2 3 4 5 ... 87 88 89 90 91 92
  * filtering                  (filtering) object 'AP band: 500 Hz high-pass;...
  * group_name                 (group_name) object 'probeA' ... 'probeA'
  * id                         (id) int64 850264150 850264158 ... 850264886
  * local_index                (local_index) int64 2 6 10 14 ... 358 362 366 370
  * location                   (location) object 'APN' 'APN' 'APN' ... '' '' ''
    ...                         ...
  * probe_vertical_position    (probe_vertical_position) int64 40 80 ... 3720
  * time                       (time) float64 28.82 28.82 ... 9.616e+03
  * valid_data                 (valid_data) bool True True True ... True True
  * x                          (x) float64 8.164e+03 8.154e+03 ... 7.125e+03
  * y                          (y) float64 3.547e+03 3.513e+03 ... 20.0 -12.0
  * z                          (z) float64 3.547e+03 3.513e+03 ... 20.0 -12.0
Data variables:
    lfp                        (time, channel) float32 dask.array<chunksize=(1000000, 20), meta=np.ndarray>
Attributes:
    reference_time:       2019-01-19T00:54:18-08:00
    session_description:  LFP data and associated channel info for a single E...
    subject_info:         {'age': 'P118D', 'genotype': 'Sst-IRES-Cre/wt;Ai32(...
    unit_attributes:      {'lfp_unit': 'volts', 'time_unit': 'seconds'}

## Creat the LFP plot

In [6]:
hover = HoverTool(tooltips=[
    ("Channel", "$name"),
    ("Time", "$x s"),
    ("Amplitude", "$y V"),
])

x_range = Range1d(start=time.min(), end=time.max())
y_range = FactorRange(factors=channels)

p = figure(x_range=x_range, y_range=y_range, active_scroll="wheel_zoom", lod_threshold=None)
p.add_tools(hover)

source = ColumnDataSource(data=dict(time=time))

for i, channel in enumerate(channels):
    xy = p.subplot(
        x_source=p.x_range,
        y_source=Range1d(start=data[i].min(), end=data[i].max()),
        x_target=p.x_range,
        y_target=Range1d(start=i, end=i + 1),
    )

    source.data[channel] = data[i]
    line = xy.line(field("time"), field(channel), color="black", source=source, name=channel)

p_line = p

## Create the probe plot

In [7]:
# Get all coordinates except 'time' into a pd df
coords_dict = {coord: ds[coord].values for coord in ds.coords if coord != 'time'}
df = pd.DataFrame(coords_dict)

# Replace NaN values in the 'location' column with string (probably above brain surface)
df['location'] = df['location'].fillna("Unknown")

unique_locations = df['location'].unique()
color_palette = viridis(len(unique_locations))

hover = HoverTool()
hover.tooltips = [(col_name, f"@{col_name}") for col_name in df.columns]

source = ColumnDataSource(df)

p = figure(width=800, height=600, title="Probe Electrodes",
           x_axis_label='Probe Horizontal Position',
           y_axis_label='Probe Vertical Position')

p.circle(x='probe_horizontal_position', y='probe_vertical_position', size=6, alpha=.5,
         fill_color=factor_cmap('location', palette=color_palette, factors=unique_locations),
         line_color="black", line_width=0.5, source=source)

p.add_tools(hover)
p.width=50

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.axis_label = None
p.yaxis.axis_label = None
p.xaxis.major_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_size = '0pt'
p.yaxis.major_label_text_font_size = '0pt'
p.outline_line_color = None
p.xaxis.axis_line_color = None
p.yaxis.axis_line_color = None

p_probe = p

## Plot probe and data

In [10]:
show(row(p_probe, p_line))

TODO: 
- Add RangeTool
- Add Annotations
- Link the probe display with traces in some way